# SW vs. HW breakpoints
SW breakpoints are basically instructions at a particular point in RAM replaced with breakpoint instruction. So it is arch specific. GDB replaces the instruction with a trap or illegal divide or some other instruction that iwll cause an exception. SW breakpoints probably, by design, won't work if code is executing off of XIP flash. 

HW breakpoints are registers that can be written to using JTAG. There are comparators inside the processor that constantly compare PC with these registers - Whenever there is a hit, it will stop execution. 

In ARM processors, the FPB unit, i.e., [Flash patch and breakpoints unit](https://developer.arm.com/documentation/ddi0337/h/debug/about-the-flash-patch-and-breakpoint-unit--fpb-) has the comparators needed for hardware breakpoints

**References**
+ [HW vs. SW interrupts](https://stackoverflow.com/questions/8878716/what-is-the-difference-between-hardware-and-software-breakpoints#:~:text=In%20brief%2C%20hardware%20breakpoints%20make,RAM%20memory%20with%20breakpoint%20instruction.)
+ [GDB breakpoint handling](https://sourceware.org/gdb/wiki/Internals/Breakpoint%20Handling)

# JTAG vs. Boundary scan vs. Trace
JTAG (Joint Test Action Group) allows user to read/write registers. It can be used for debugging or testing some SoC functionality without having to use the CPU in the SoC. Boundary scan feature is used by hardware test tools to find if a particular IC part is good or bad. The Embedded Trace Macrocell (ETM) interface enables you to connect an external ETM unit 9for example XDS560V2 Pro) to the processor for real-time code tracing of the core. JTAG signals are also required for ETM trace.

**From a random guy on the internet**:

trace is the record of a instruction sequence what happend before the
trigger point (e.g. a breakpoint). With other words, a trace feature is
usefull if a breakpoint triggers and you like to see what reasons were
therefore and why the CPU arrived at the breakpoint location.

Some JTAG CPUs have seperate connectors (e.g. OCDS Level1 and Level2) to
record the instructions by data and addressbus with external trace memory.
Other embedded CPUs have internal trace fifo
(e.g. Motorolas 9S8) what can be read out over JTAG or BDM or
similar Interfaces even while CPU is running. Some of the new controllers
do not have a test mode what allows to access external bus lines and even
the special bondout chips are very expensive and make many trouble for
emulator manufacturers. The benefit of external trace is the bandwidth
combined with complex trigger conditions but for most problems simple
internal trace memory will do the job why I think this will be the future.

**A different random guy on the internet**:

Departing from PowerPC ARM's trace solution makes use of an Embedded Trace
Macrocells which essentially snoops on the core an outputs a highly
compressed stream which can be reconstructed into an historical record of
the execution of the processor and memory transfers.

This is exceedingly useful for debugging several classes of problems,
particularly ones which do not occur when a conventional debugger is
attached due to the debugger changing the timing characteristics of the
system.

The ETM also includes powerful filtering capabilities to for trace port
bandwidth conservation.

Some PowerPC processors have a system called NEXUS which is based on the
same general idea but most of the implementations I know of aren't as
powerful as the ETM based solution see http://www.nexus5001.org/ for
details.

Since trace is non invasive it can also be used for non invasive profiling
of release code.

# Simulator vs. Emulator
In the context of a microcontroller, a simulator uses a PC to simulate the execution of a code in the microcontroller. Processor read writes to register, memory are simulated in software. But external stimulus is modeled in a file (which has samples of that stimulus). The simulator could be made clock accurate, but not time accurate (it will take the same number of clocks as a mC would, but in terms of time, it could take more or less).

Emulators use ICE/JTAG to basically let you load (modified code) easily into say some SRAM from which it is executed. The user can then use breakpoints, add register to watchlist, read to regs, write to regs (actual mCs regs). 

# FPGA vs CPLD
PLD has logic blocks with fuses interconnecting gates that can be blown (or not blown) to achieve required combinational logic circuit. CPLDs have several PLD blocks, where the connections between the PLDs themselves is configurable. CPLDs can be thought of configurable PCB with with several 74xx series logic ICs. Original PLDs were not reconfigurable and were based on ROMs. Modern ones are reconfigurable - they are based out of EEPROM. Many can be reconfigured several hundred times. CPLDs retain programmed logic through power cycle and will start functioning right after power up.

FPGAs have much finer logic blocks that CPLDs and hence more customizable than CPLDs. They are based on RAMs. FPGAs do not retain their configuration and hence require time to configure themselves using configuration stored in a flash. This means they can start working right after power up - like uCs they may take several seconds to boot up and become functional. FPGA can be thought of as reconfigurable silicon cores rather than reconfigurable PCBs. FPGAs may have fixed function blocks such as multipliers, RAMs, PCIE controllers etc.

# EERAM
EERAM = EEPROM + SRAM in one chip with some power loss detection and data backup circuitry. 

**Microchip marketing material**:
EERAM is a serial SRAM product that has hidden nonvolatile bits. On any power disruption, the active SRAM array content is moved to the nonvolatile bits.  When power is restored, the nonvolatile content is restored to the SRAM array and SRAM operation continues

+ Unlimited Reads/Unlimited Writes: Standard SRAM cells and protocol with symmetrical read and write timing
+ Industry-Standard Bus Types: SPI, I2C
+ Invisible-to-User Data Transfers on Power Loss: Hidden nonvolatile cells capture SRAM content; power loss is detected automatically by the EERAM device
+ 100,000 Backups: Data can be moved from SRAM to nonvolatile cells > 100,000 times
+ 100-Year Data Retention: With power removed, data is kept safe for up to 100 years; when power is restored, data is moved back to SRAM for continuing operation
+ Quality: EERAM is based on high-volume, standard CMOS processes to create a very low-cost nonvolatile SRAM solution and the highest quality levels

# Power management
In Tegra, Power and Boot management were put together in one subsystem because when power comes up, the power-on-reset circuit needs to be on the rail that PMIC always turns on by default. And right after power-on-reset hardware, comes booting process. Furthermore, when boot is done, the subsystem would be useless if it isn't doing some power management functions - like DVFS, clock/power gating, turning on/off rails etc. While going into sleep state, the power management has to do required clock/power gating, saving of states etc., in a particular sequence and repeat in the reverse during wake up. But during the sleep, the power management module still needs to be awake to be able to process wake interrupts.

# C stuff
## Difference between stdout and stderr
stdout is buffered while stderr isn't. What that means is, a printf to stdout may not occur immediately in the terminal, while a fprintf to stderr will. It is also possible to send printf output to a file, but stderr will always print on the terminal. So it is better for displaying critical errors.

## Type qualifiers
  + const, volatile, restrict are type qualifiers
  + const is better than #define in the sense that it can be treated like other variables - Mainly, its scope is restricted to the function it is inside
  + Volatile tells the compiler that variables value can change in ways that cannot be understood by looking at the code alone. It is useful if there a variable (global variable/semaphore etc.) can be changed by say an ISR or a different thread or a different processor core. For ex. if a global integer a=0; is defined at the top of a file and elsewhere there is a if/else where a==0 is done, without volatile qualifier, the compiler would do away with the if/else entirely. However, a context switch to a different thread or to an ISR may occur between the declaration and the if/else and a's value might change. This is where volatile is useful
  + restrict qualifier to a pointer tells the compiler that the object pointed to is not referenced by any other pointer - This may be useful to the compiler while it is doing optimization. But if, in the code, somehow aanother pointer points to the same object, compiler will *not* throw any error. 
  
## Storage classes
  + auto, extern, static, register are C storage classes
  + auto is the default. It is valid only for local variables. There is no difference between int a; and auto int a;
  + extern is for global variables defined outside in the file in question
  + register tells the compiler to *better* store the variable in a processor register rather than a RAM. But there is no guarantee compiler will do that. One important thing is that since a register can store only one word, one can use it for data types that have a width less than or equal to one word. 
  + static has two uses: It defines the scope of the variable/function to within a file it is defined in. When static is used inside a function for a local variable, it means its value won't be lost between function calls. static local variables are stored in .data (if initialized) or .bss (if uninitialized) just like global variables, but their scope is within the funciton in which they are defined.

## GCC stuff
  + gcc -Wall options displays all warnings
  + gcc -Wall -Werror converts warnings to errors
  + gcc -printsize option prints a table showing the sizes of .text, .data, .bss, .interrupts (IVT), .user_heap_stack areas.
  
### Makefile
Language agnostic. It runs some shell scripts based on target name supplied by the user. Ex. In 'makefile' (no extensions), we can have:

In [ ]:
CC=gcc
CFLAGS = -Wall -WError

all: hellomain.o hellofun.o
    $(CC) hellomain.o hellofun.o -o hello
 
hellomain.o: hellomain.c hellofun.h
    $(CC) $(CFLAGS) -c hellomain.c
 
hellofun.o: hellofun.c hellofun.h
    $(CC) $(CFLAGS) -c hellofun.c
 
clean:
    rm -rf *.o
    rm hello

When user does \\$make, the default target is chosen, which is the topmost target, i.e., *all*. Otherwise the user will have to specify, like \\$make clean. Based on the chosen target, make utility will look at dependencies of that target. So, for all, we need hellomain.o and hellofun.o to be present. These themselves are targets, so we need to basically execute them one by one. 

Here is another example:

In [ ]:
blah: blah.o
    gcc blah.o -o blah # Runs third

blah.o: blah.c
    gcc -c blah.c -o blah.o # Runs second

blah.c:
    echo "int main() { return 0; }" > blah.c # Runs first

In this example:
  + Make is given blah as the target, so it first searches for this target
  + blah requires blah.o, so make searches for the blah.o target
  + blah.o requires blah.c, so make searches for the blah.c target
  + blah.c has no dependencies, so the echo command is run
  + The cc -c command is then run, because all of the blah.o dependencies are finished
  + The top cc command is run, because all the blah dependencies are finished
  + That's it: blah is a compiled c program

# Logic Analyzers
Used to probe 1s and 0s on I2C, SPI, UART etc., They sample the analog signal on the line and convert them to 1s or 0s. They do not show what is actually happening on the line like scopes do - the line may impulse noises here and there, which may be interpreted by the system it is hooked up to as 1s - the logic analyzer may miss them altogether. 

Logic analyzer specifications inlcude the number of channels, sampling frequency, how much buffer it has to store sampled data, can it accept external clock or not, does it have in-buit compression, is it real time (uploads data to say a PC monitor as soon as it get it, or store data in internal buffer and then once it is done, upload it to the computer). 

Just as with scopes, with logic analyzers, it is all about the analog front end: Noise, skews across channels, input capacitance, inductance, rise time, fall time, metastability issues etc. If you are pulling out a logic analyzer it means you are desperate. Either nothing is working, or everything is working 99% of the time, and you want to know why it is failing 1% of the time. There may be some tiny timing issue (often the logic analyzer is needed only while working with high speed protocols). Just the act of hooking up the logic analyzer will have an impact on the signal characteristics on a line. Costly analyzers have good AFEs and affect the lines less.  

Good logic analyzers have triggering facility where you can set a bit-string pattern and the analyzer will capture x bits of data that occurred before or after the trigger. It is basically like the digital version of a scope's trigger. Good logic analyzers also have low, high threshold levels (for deciding 0 vs. 1)

Some logic analyzers go one step beyond and parse the logic output and show human readable messages, highlight protocol errors and so on. 

### References
[1] https://www.circuitstoday.com/understanding-fpga-and-cpld
[2] https://numato.com/kb/cpld-vs-fpga-differences-one-use/ 